<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/Regression_DaysBurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As the classification models have had some difficulties classifying the fires accurately, we will attempt to build regression models (on Days Burn feature to see if models can be built on this) instead and see if these have more success.

I will deploy the following techniques:

1.   One-Hot Encoding of Categorical Variables
2.   Splitting the Training and Testing data
3.   Feature Selection
4.   Normalize Data
5.   Regressions




In [530]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#Read in the smaller of the final datasets. The dataset contains around 1200 fires from 2016-2019
wildfire_set1 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/wildfire_set1_w_allfeatures.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [531]:
wildfire_set1.head()

,Unnamed: 0,Year,Name,AcresBurned,Fire Size Rank,Cause,SOURCE_REPORTING_UNIT_NAME,DaysBurn,Discovery Month,Discovered DOY,...,DX90_2M,DP10_2M,Receives Snow,Snow Station,River Basin,Mar_SP,Mar_WC,Mar_Dens,Has_Elevation,Elevation
0,0,2005,FOUNTAIN,0.10,A,Miscellaneous,Plumas National Forest,1.0,Feb,33.0,...,0.0,19.0,1.0,eureka_lake,Feather,79.6,34.0,0.43,1,2965.8432
1,1,2004,PIGEON,0.25,A,Lightning,Eldorado National Forest,1.0,May,133.0,...,0.0,3.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35,1,6207.2736
2,2,2004,SLACK,0.10,A,Debris Burning,Eldorado National Forest,1.0,Jun,152.0,...,0.0,11.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35,1,3454.6824
3,3,2004,DEER,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33,1,7759.0920
4,4,2004,STEVENOT,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33,1,7598.3328


In [532]:
wildfire_set1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114558 entries, 0 to 114557
Data columns (total 40 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  114558 non-null  int64  
 1   Year                        114558 non-null  int64  
 2   Name                        103308 non-null  object 
 3   AcresBurned                 114558 non-null  float64
 4   Fire Size Rank              114558 non-null  object 
 5   Cause                       114558 non-null  object 
 6   SOURCE_REPORTING_UNIT_NAME  114558 non-null  object 
 7   DaysBurn                    61574 non-null   float64
 8   Discovery Month             114558 non-null  object 
 9   Discovered DOY              114558 non-null  float64
 10  Contained Month             61574 non-null   object 
 11  Contained DOY               61574 non-null   float64
 12  Latitude                    114558 non-null  float64
 13  Longitude     

In [533]:
print(wildfire_set1.columns)

Index(['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Fire Size Rank', 'Cause',
       'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month',
       'Discovered DOY', 'Contained Month', 'Contained DOY', 'Latitude',
       'Longitude', 'County', 'CountyIds', 'State', 'OWNER_DESCR',
       'NOAA Station', 'Link', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP', 'TAVG',
       'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Snow Station', 'River Basin', 'Mar_SP', 'Mar_WC',
       'Mar_Dens', 'Has_Elevation', 'Elevation'],
      dtype='object')


Drop all columns that will not be relevant for our regression task:Unnamed: 0, Year, Name, Cause, SOURCE_REPORTING UNIT NAME, AcresBurned, Discovered DOY, Contained Month, Contained DOY, Lat, Long, County Ids, State, OWNER_DESCR, NOAA Station, Link, Snow Station, River Basin

In [534]:
wildfire_set1.drop(columns = ['Unnamed: 0', 'AcresBurned', 'Year', 'Name', 'Cause', 'Fire Size Rank',
                      'SOURCE_REPORTING_UNIT_NAME', 'Contained Month', 'Discovered DOY',
                      'Contained DOY','Latitude','Longitude','County','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin','Has_Elevation'], inplace = True)

Taking a look at the dataset now that we have dropped the nonrelevant columns

In [535]:
wildfire_set1.head(5)

,DaysBurn,Discovery Month,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens,Elevation
0,1.0,Feb,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,49.06,0.0,19.0,1.0,79.6,34.0,0.43,2965.8432
1,1.0,May,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,14.76,0.0,3.0,1.0,108.6,38.1,0.35,6207.2736
2,1.0,Jun,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,40.37,0.0,11.0,1.0,108.6,38.1,0.35,3454.6824
3,5.0,Jun,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33,7759.0920
4,5.0,Jun,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,14.76,0.0,3.0,1.0,87.2,28.4,0.33,7598.3328


In [536]:
wildfire_set1.shape

(114558, 19)

# Investigate the presence of null values in the dataset:
Below, we have the count of totall null values in each column for  wildfire_set1.

In [537]:
print(wildfire_set1.isnull().sum())

DaysBurn           52984
Discovery Month        0
AWND                6254
CLDD                3335
DP10                3042
DX90                3333
PRCP                3042
TAVG                3335
TMAX                3333
TMIN                3285
PRCP_6M             2470
PRCP_RS             2811
DX90_2M             2215
DP10_2M             1599
Receives Snow        102
Mar_SP               102
Mar_WC               102
Mar_Dens             102
Elevation              0
dtype: int64


In [538]:
wildfire_set1 = wildfire_set1.dropna()

In the cell above, we drop all null values in the dataset. We are able to drop all nulls in this case because our dataset is sizeable with relatively few null values in each so not much data is lost in this process. Looking at the shape of the dataframe after the drops, we see that the dataset remains a similar size post drop.

In [539]:
#we lose around 55,000 values when dropping NaN's
wildfire_set1.shape

(57172, 19)

# For our regressions, we will try to predict the number of Days the fire burns

In [540]:
print(wildfire_set1.columns)

Index(['DaysBurn', 'Discovery Month', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP',
       'TAVG', 'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Mar_SP', 'Mar_WC', 'Mar_Dens', 'Elevation'],
      dtype='object')


Double check our datatypes before we proceed with preprocessing and model building. As we can see, all features besides discovery and fire size rank are numerical (of type float or int) so all we have to do is one hot encode the discovery month of the fire.

In [541]:
wildfire_set1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57172 entries, 0 to 114553
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DaysBurn         57172 non-null  float64
 1   Discovery Month  57172 non-null  object 
 2   AWND             57172 non-null  float64
 3   CLDD             57172 non-null  float64
 4   DP10             57172 non-null  float64
 5   DX90             57172 non-null  float64
 6   PRCP             57172 non-null  float64
 7   TAVG             57172 non-null  float64
 8   TMAX             57172 non-null  float64
 9   TMIN             57172 non-null  float64
 10  PRCP_6M          57172 non-null  float64
 11  PRCP_RS          57172 non-null  float64
 12  DX90_2M          57172 non-null  float64
 13  DP10_2M          57172 non-null  float64
 14  Receives Snow    57172 non-null  float64
 15  Mar_SP           57172 non-null  float64
 16  Mar_WC           57172 non-null  float64
 17  Mar_Dens   

In [542]:
#Drop any incorrect values for DaysBurn where it is a negative number
for index, row in wildfire_set1.iterrows():
  if wildfire_set1.at[index, 'DaysBurn'] < 0:
     wildfire_set1.drop(labels = index, axis = 0, inplace = True)

In [543]:
wildfire_set1.shape

(57170, 19)

# Below we use the describe function to get a general outlook on mean, max, min, and percentiles for each of the numerical features in the dataset

In [544]:
wildfire_set1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
DaysBurn,57170.0,1.791184,6.845498,1.0000,1.0000,1.0000,1.0000,273.0000
AWND,57170.0,6.223038,1.855344,0.4000,4.9000,5.8000,7.2000,17.2000
CLDD,57170.0,233.095802,229.053031,0.0000,9.0000,182.0000,401.0000,1113.0000
DP10,57170.0,0.985359,1.850075,0.0000,0.0000,0.0000,1.0000,24.0000
DX90,57170.0,12.223544,11.922190,0.0000,0.0000,9.0000,24.0000,31.0000
PRCP,57170.0,0.487439,1.176507,0.0000,0.0000,0.0700,0.4400,20.5900
TAVG,57170.0,69.165659,11.582415,18.3000,59.9000,70.5000,78.2000,100.9000
TMAX,57170.0,82.945216,13.844788,33.0000,72.6000,84.9000,94.4000,114.3000
TMIN,57170.0,55.385783,10.424397,3.7000,48.5000,56.1000,62.8000,89.0000
PRCP_6M,57170.0,9.916725,10.305508,0.0000,2.5200,6.3800,14.1200,85.9100


# Some Interesting Takeaways:
*   Acres Burned has an average of 5200 acres yet the 50th percentile fire only has 87 acres burned. This tells us there are some extremely massive fires in this dataset contributing to this large mean acres burned
*   DP10 and PRCP look to not be the best distinguishing features as 0-50th percentile for both all have 0. This can be attributed to the vast majority of the state of California having undergone extreme drought conditions during the last decade (with some exceptions in rainy coastal areas in Northern California)
*  The temperature features seem to have a wide spread which is good
*  Three of our engineered weather features DX90_2M, PRCP_6M, PRCP_RS seem to have a healthy spread and diverse values which is good
*   Average Elevation that fires in the dataset occurred at is 1760 ft. with 75% occuring under 2398 ft. Interesting to see that most fires begin at lower elevations







# One-Hot Encoding Categorical Variables


*   Encode the discovery month (the only categorical variable) in both datasets



In [545]:
#Add the dummy columns to the dataset
dummy_month = pd.get_dummies(wildfire_set1['Discovery Month'])
wildfire_set1 = pd.merge(left = wildfire_set1, right = dummy_month, left_index = True, right_index = True)
wildfire_set1 = wildfire_set1.drop(columns = ['Discovery Month'])
wildfire_set1.head(20)

,DaysBurn,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,1.0,5.6,0.0,12.0,0.0,5.33,38.9,43.9,33.9,27.89,...,0,1,0,0,0,0,0,0,0,0
1,1.0,6.9,0.0,2.0,0.0,0.81,47.3,63.0,31.6,14.37,...,0,0,0,0,0,0,1,0,0,0
2,1.0,5.6,36.0,0.0,0.0,0.00,63.1,70.2,56.0,36.71,...,0,0,0,0,1,0,0,0,0,0
3,5.0,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
4,5.0,5.6,0.0,1.0,0.0,0.29,54.7,72.9,36.5,13.63,...,0,0,0,0,1,0,0,0,0,0
5,1.0,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0
6,1.0,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0
7,1.0,6.5,7.0,9.0,0.0,4.99,55.5,67.0,44.0,25.91,...,0,0,0,0,0,1,0,0,0,0
8,1.0,6.5,7.0,9.0,0.0,4.99,55.5,67.0,44.0,25.91,...,0,0,0,0,0,1,0,0,0,0
9,1.0,5.1,1.0,0.0,0.0,0.00,61.1,79.8,42.3,8.31,...,0,0,0,1,0,0,0,0,0,0


In [546]:
#Radomly Shuffle the dataframe to get rid of any pre-existing organization based on size
wildfire_set1 = wildfire_set1.sample(frac=1).reset_index(drop=True)

In [547]:
wildfire_set1.head()

,DaysBurn,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,1.0,4.7,149.0,0.0,0.0,0.01,69.8,75.5,64.1,8.34,...,0,0,0,1,0,0,0,0,0,0
1,1.0,7.2,941.0,2.0,31.0,0.58,95.4,106.7,84.0,2.99,...,0,0,0,0,0,0,0,0,0,0
2,1.0,5.1,32.0,1.0,2.0,0.29,63.6,76.2,51.0,0.12,...,0,0,0,0,0,0,0,1,0,0
3,1.0,6.3,805.0,0.0,31.0,0.00,91.0,106.4,75.5,0.11,...,0,0,0,0,0,0,0,0,0,0
4,1.0,4.9,10.0,3.0,2.0,1.62,60.7,77.7,43.6,0.62,...,0,0,0,0,0,0,0,0,1,0


In [548]:
#Write the clean, encoded, and randomly shuffled data to csv before continuing 
wildfire_set1.to_csv('wildfire1_regression_clean.csv')

Split the data into train-test sets

In [549]:
#We will train our models using the more recent dataset

In [550]:
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [551]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay

Utilize an 80-20 train test split on our large dataset

In [552]:
train_data, test_data = train_test_split(wildfire_set1, test_size = 0.2, random_state = 1)

Print the shape of training and testing datasets after the split to make sure we have done this correctly

In [553]:
print(train_data.shape)
print(test_data.shape)

(45736, 30)
(11434, 30)


# Feature Selection

In [554]:
from sklearn import linear_model

In [555]:
from sklearn.feature_selection import SelectKBest, f_regression

In [556]:
top_features_ordered = []

In [557]:
train_target = train_data['DaysBurn']
train_features = train_data.drop(columns = ['DaysBurn'])

In [558]:
test_target = test_data['DaysBurn']
test_features = test_data.drop(columns = ['DaysBurn'])

Select the 10 best features using sklearn's SelectKBest

In [571]:
best_features = SelectKBest(f_regression, k=12)
k_best_data = best_features.fit_transform(train_features, train_target)
mask = best_features.get_support()
new_features = train_features.columns[mask]
for item in new_features:
  if item not in top_features_ordered:
    top_features_ordered.append(item)

In [572]:
count = 1
for item in top_features_ordered:
  print(count, '. ', item)
  count += 1

1 .  CLDD
2 .  DX90
3 .  TAVG
4 .  TMAX
5 .  Receives Snow
6 .  Mar_SP
7 .  Mar_WC
8 .  Mar_Dens
9 .  Elevation
10 .  Jul
11 .  DX90_2M
12 .  May


# Now we normalize the train and test sets independently

In [573]:
train_data_selected_X = train_features[top_features_ordered]
train_data_y = train_target

test_data_selected_X = test_features[top_features_ordered]
test_data_y = test_target

In [574]:
print('Train Features:')
print('train_data_X dimensions: ', train_data_selected_X.shape)
print('train_data_y dimensions:', train_data_y.shape, '\n')

print('Test Features:')
print('test_data_X dimensions: ', test_data_selected_X.shape)
print('test_data_y dimensions:', test_data_y.shape, '\n')

Train Features:
train_data_X dimensions:  (45736, 12)
train_data_y dimensions: (45736,) 

Test Features:
test_data_X dimensions:  (11434, 12)
test_data_y dimensions: (11434,) 



Normalize data using min-max-scalar (every feature will normalized so that range of values is 0-1) after splitting into train and test

In [575]:
from sklearn.preprocessing import MinMaxScaler

In [576]:
#utilize minmax scaler normalization
scaler = MinMaxScaler()

#Normalize the train, validate, and test independently after splitting
train_data_X = scaler.fit_transform(train_data_selected_X)
#train_target = scaler(train_target)

test_data_X = scaler.fit_transform(test_data_selected_X)
#train_target = scaler(test_target)

# Fit the model using our normalized features to predict Acres Burned

In [577]:
multivariate_regression = linear_model.LinearRegression()
multivariate_regression.fit(train_data_X, train_data_y)

LinearRegression()

Model coefficients: (Beta's)

In [578]:
multivariate_regression.coef_

array([-5.37195778,  3.15324102,  6.64030033, -5.74652926,  0.47050449,
       -7.28913023, 10.12160009, -2.40947615,  5.44348528,  0.27360529,
       -0.32230573, -0.14649318])

Model intercept:

In [579]:
multivariate_regression.intercept_

-0.11086561450110088

In [580]:
from sklearn import metrics
import numpy as np

In [581]:
y_predicted = multivariate_regression.predict(test_data_X)

In [582]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_predicted), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_predicted), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_predicted)))

Mean Squared Error:  43.839222131637015 

Mean Absolute Error:  1.5541250076876278 

Root Mean Squared Error:  6.621119401705199
